In [1]:
from openai import OpenAI
import os
import time
import json

import openpyxl
import pandas as pd
import csv

client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
# print(OpenAI.api_key)

In [17]:
workbook = openpyxl.load_workbook('data\ExampleFunctionCall.xlsx')
sheet = workbook['Testset']
new_sheet = workbook['A']

In [19]:
for i in range(2,211):
    if sheet['D'+str(i)].value == "advice":
        data_to_copy = []
        for cell in sheet[i]:
            data_to_copy.append(cell.value)
        new_sheet.append(data_to_copy)


In [20]:
workbook.save('data\ExampleFunctionCall.xlsx')

In [ ]:
for i in range(2,623):
    if(not sheet['C'+str(i)].value.startswith("function_")):
        # print(i)
        sheet['E'+str(i)].value = "direct answer"

In [ ]:
for i in range(2,623):
    if(sheet['B'+str(i)].value == "function_6"):
        print(i)

In [ ]:
workbook.save('QAK0620.xlsx')

In [ ]:
instruction = """
    # Objective:
    As an intelligent customer service chatbot of the “Flora” focus app , my goal is to understand and help you solve problems encountered when using flora app effectively.
    # Main features of the Flora app:
    Flora is a new way to stay off your phone, clear to-do lists, and build positive, life-changing habits. Whenever you want to make progress towards your goals, plant a seed in Flora. As you work hard, the seed will grow into a healthy tree. Let the tree be your coach and grow yourself with it. You will be amazed by how great you can be. FOCUS TREES Have trouble putting down the phone? Flora blocks distracting apps in a pleasant way to help you focus on what’s more important in real life. If you cannot resist the temptation of using your phone and leave the app while growing a tree, the tree will be killed! But if succeeding, you'll unlock new trees. DOUBLE EFFICIENCY WITH FRIENDS Things get even more interesting when you challenge your friends to plant trees together—you will see who kills a tree (ouch). However, if you successfully stay away from your phones together, each of you will win an additional tree from a random friend. You can also chat and share your progress to motivate each other and keep everyone on track.* * Flora is the original app for multi-user tree planting and delivers the best features that help you and your friends be productive. TO-DO LIST & WIDGET Flora is a to-do list, and more. Simply create a to-do item and set a reminder, then you will never forget a thing. Even better, you can tag your trees with a to-do item to easily track the progress towards each of your life goals. No matter if your goal is daily, weekly, or monthly, Flora will help you follow it through and achieve it. HABIT TRACKER Enjoy a rewarding daily habit routine. If you grow trees regularly, you will earn yourself a beautiful, vibrant garden. Take a look at it and feel proud of yourself! Each tree records your perseverance and a step towards a healthy life. You can also view the daily, weekly, or monthly statistics of your activities in Flora to find opportunities to improve your time-management and planning skills. REAL TREES, REAL IMPACT Why not plant REAL trees while improving yourself? Flora proudly offers the opt-in Price and Care services that let you plant real trees on the earth if you kill or successfully grow a tree in the app. We partner with tree-planting organizations in Africa and East Asia, such as the Trees.org and Eden Reforestation Projects, to plant fast-growing soil rebuilding trees, fruit trees that diversify incomes and nutrition, and trees that can provide forage and fuel-wood. All these trees help make the earth greener and provide long-term financial aid to families who need it most.
    # Support Process:
    Issue Identification: To ensure that I understand your problems, I will summarize the constructive, non-emotional parts of our input text in your tone into one key point. but if one key point is not enough to describe problem, key point can be more than one. Avoid using conjunctions. Then, I will and must only output these key points in json format.
    # Example:
    input: How do I change my password?
    output: {"keypoints": ["my problem is I need to change password."]}
    input: I am just doing my a-levels at the moment and I love using your app for revision. However, when I went onto the app earlier it logged me out and all my data had been lost. I was just wondering if you could help me out.
    output: {"keypoints": ["My data lost after being logged out of the app."]}
    input: My flora friend page won't load. If I delete the app and re download it and then login. Will all my data and garden time be lost?
    output: {"keypoints": ["my Flora friend page will not load.","I am concern about losing data and garden time after deleting and reinstalling the app."]}
"""

print(instruction)

In [ ]:
with open('QAK0620.csv') as file:
    reader = csv.reader(file)
    data = list(reader)

In [ ]:
def is_json(myjson):
    try:
        json.loads(myjson)
    except ValueError:
        return False
    return True

for i in range(2,624):
    if not is_json(data[i][2]):
        print(i)

In [ ]:
for i in range(669,670):

    question = data[i][0]

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": instruction},
            {"role": "user", "content": question}
        ]
    )
    time.sleep(3)
    ##data[i].append(completion.choices[0].message.content.replace("\n", " "))
    keyjson = completion.choices[0].message.content
    if is_json(keyjson):
        data[i].append(completion.choices[0].message.content.replace("\n", " "))
    else:
        data[i].append('*')


In [ ]:
with open('./data/QAK.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

In [ ]:
df = pd.read_csv("QAK0620.csv")
print(df)

In [2]:
def get_text_embeddings(csv_file, max_length=8191):
    df = pd.read_csv(csv_file)
    QP = []
    embeddings = []

    for index, row in df.iterrows():

        question = row['Question']
        key_point = row['Key_Point']
        key_str = ""
        for key in json.loads(key_point)['keypoints']:
            key_str = key_str+key

        # Concatenate question and key point strings
        concatenated_text = f"{question}{key_str}"

        # Split the text into tokens
        tokens = concatenated_text.split()

        # Truncate or pad the tokens to the maximum length
        tokens = tokens[:max_length]

        # Join the tokens back into a single string
        truncated_text = ' '.join(tokens)

        # Get the text embeddings using OpenAI
        embedding = client.embeddings.create(input=[truncated_text], model="text-embedding-3-large", dimensions=3072).data[0].embedding
        time.sleep(3)

        # Append the embedding to the list
        QP.append(concatenated_text)
        embeddings.append(embedding)

    # Create a new DataFrame with the embeddings
    QP_df =  pd.DataFrame(QP)
    embeddings_df = pd.DataFrame(embeddings)

    return embeddings_df

In [3]:
df_QP, df_embeddings = get_text_embeddings('QAK0620.csv')
df_QP.to_csv('./data/QP0620.csv', index=False)
df_embeddings.to_csv('./data/QK_embeddings0620.csv', index=False)